<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_11_03_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks
**Module 11: Natural Language Processing and Speech Recognition**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 11 Material

* Part 11.1: Getting Started with Spacy in Python [[Video]](https://www.youtube.com/watch?v=A5BtU9vXzu8&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_01_spacy.ipynb)
* Part 11.2: Word2Vec and Text Classification [[Video]](https://www.youtube.com/watch?v=nWxtRlpObIs&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_02_word2vec.ipynb)
* **Part 11.3: What are Embedding Layers in Keras** [[Video]](https://www.youtube.com/watch?v=OuNH5kT-aD0&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_03_embedding.ipynb)
* Part 11.4: Natural Language Processing with Spacy and Keras [[Video]](https://www.youtube.com/watch?v=BKgwjhao5DU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_04_text_nlp.ipynb)
* Part 11.5: Learning English from Scratch with Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=Y1khuuSjZzc&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=58) [[Notebook]](t81_558_class_11_05_english_scratch.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.

In [1]:
try:
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: not using Google CoLab


# Part 11.3: What are Embedding Layers in Keras

[Embedding Layers](https://keras.io/layers/embeddings/) are a handy feature of Keras that allows the program to automatically insert additional information into the data flow of your neural network.  In the previous section, you saw that Word2Vec could expand words to a 300 dimension vector.  An embedding layer would allow you to insert these 300-dimension vectors in the place of word-indexes automatically.  

Programmers often use embedding layers with Natural Language Processing (NLP); however, they can be used in any instance where you wish to insert a lengthier vector in an index value place.  In some ways, you can think of an embedding layer as dimension expansion. However, the hope is that these additional dimensions provide more information to the model and provide a better score.

### Simple Embedding Layer Example

* **input_dim** = How large is the vocabulary?  How many categories are you encoding? This parameter is the number of items in your "lookup table."
* **output_dim** = How many numbers in the vector that you wish to return. 
* **input_length** = How many items are in the input feature vector that you need to transform?

Now we create a neural network with a vocabulary size of 10, which will reduce those values between 0-9 to 4 number vectors.  Each feature vector coming in will have two such features.  This neural network does nothing more than pass the embedding on to the output.  But it does let us see what the embedding is doing.

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import numpy as np

model = Sequential()
embedding_layer = Embedding(input_dim=10, output_dim=4, input_length=2)
model.add(embedding_layer)
model.compile('adam', 'mse')

Let's take a look at the structure of this neural network so that we can see what is happening inside it.

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 4)              40        
Total params: 40
Trainable params: 40
Non-trainable params: 0
_________________________________________________________________


For this neural network, which is just an embedding layer, the input is a vector of size 2. These two inputs are integer numbers from 0 to 9 (corresponding to the requested input_dim quantity of 10 values). Looking at the summary above, we see that the embedding layer has 40 parameters.  This value comes from the embedded lookup table that contains four amounts (output_dim) for each of the 10 (input_dim) possible integer values for the two inputs. The output is 2 (input_length) length 4 (output_dim) vectors, resulting in a total output size of 8, which corresponds to the Output Shape given in the summary above.

Now, let us query the neural network with two rows.  The input is two integer values, as was specified when we created the neural network.

In [4]:
input_data = np.array([
    [1,2]
])

pred = model.predict(input_data)

print(input_data.shape)
print(pred)

(1, 2)
[[[ 0.00050902 -0.0099237  -0.02883428 -0.00821529]
  [ 0.02421514  0.03112716  0.02453538 -0.01354214]]]


Here we see two length-4 vectors that Keras looked up for each of the input integers.  Recall that Python arrays are zero-based.  Keras replaced the value of 1 with the second row of the 10 x 4 lookup matrix.  Similarly, Keras replaced the value of 2 by the third row of the lookup matrix. The following code displays the lookup matrix in its entirety. The embedding layer performs no mathematical operations other than inserting the correct row from the lookup table.

In [5]:
embedding_layer.get_weights()

[array([[ 3.9767805e-02,  1.3598096e-02, -1.1770856e-02, -3.6321927e-02],
        [ 5.0902367e-04, -9.9236965e-03, -2.8834283e-02, -8.2152858e-03],
        [ 2.4215136e-02,  3.1127159e-02,  2.4535384e-02, -1.3542138e-02],
        [ 4.6041872e-02,  4.6050500e-02,  2.2079099e-02, -4.1100323e-02],
        [-1.9084716e-02,  1.5681457e-02,  1.9137934e-04,  1.2393482e-03],
        [ 5.9496611e-05, -4.3054130e-02,  3.0203927e-02, -3.3005968e-02],
        [ 1.4646780e-02, -2.4961460e-02, -8.0889687e-03,  3.7561730e-04],
        [ 3.7584487e-02, -3.7326049e-02,  3.9304320e-02, -4.2805064e-02],
        [ 1.3853524e-02, -1.7934263e-02, -4.2281806e-02, -3.8661052e-02],
        [-4.2616617e-02,  1.4965128e-02, -3.5379924e-02, -3.9788373e-03]],
       dtype=float32)]

The values above are random parameters that Keras generated as starting points.  Generally, we will either transfer an embedding or train these random values into something useful.  The next section demonstrates how to embed a hand-coded embedding. 

### Transferring An Embedding

Now, we see how to hard-code an embedding lookup that performs a simple one-hot encoding.  One-hot encoding would transform the input integer values of 0, 1, and 2 to the vectors $[1,0,0]$, $[0,1,0]$, and $[0,0,1]$ respectively. The following code replaced the random lookup values in the embedding layer with this one-hot coding inspired lookup table.

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import numpy as np

embedding_lookup = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1]
])

model = Sequential()
embedding_layer = Embedding(input_dim=3, output_dim=3, input_length=2)
model.add(embedding_layer)
model.compile('adam', 'mse')

embedding_layer.set_weights([embedding_lookup])

We have the following parameters to the Embedding layer:
    
* input_dim=3 - There are three different integer categorical values allowed.
* output_dim=3 - Per one-hot encoding, three columns represent a categorical value with three possible values.
* input_length=2 - The input vector has two of these categorical values.

Now we query the neural network with two categorical values to see the lookup performed.

In [7]:
input_data = np.array([
    [0,1]
])

pred = model.predict(input_data)

print(input_data.shape)
print(pred)

(1, 2)
[[[1. 0. 0.]
  [0. 1. 0.]]]


The given output shows that we provided the program with two rows from the one-hot encoding table. This encoding is a correct one-hot encoding for the values 0 and 1, where there are up to 3 unique values possible. 

The next section demonstrates how to train this embedding lookup table.

### Training an Embedding

First, we make use of the following imports.

In [8]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense

We create a neural network that classifies restaurant reviews according to positive or negative.  This neural network can accept strings as input, such as given here.  This code also includes positive or negative labels for each review.

In [9]:
# Define 10 resturant reviews.
reviews = [
    'Never coming back!',
    'Horrible service',
    'Rude waitress',
    'Cold food.',
    'Horrible food!',
    'Awesome',
    'Awesome service!',
    'Rocks!',
    'poor work',
    'Couldn\'t have done better']

# Define labels (1=negative, 0=positive)
labels = array([1,1,1,1,1,0,0,0,0,0])

Notice that the second to the last label is incorrect.  Errors such as this are not too out of the ordinary, as most training data could have some noise.

We define a vocabulary size of 50 words.  Though we do not have 50 words, it is okay to use a value larger than needed.  If there are more than 50 words, the least frequently used words in the training set are automatically dropped by the embedding layer during training.  For input, we one-hot encode the strings.  Note that we use the TensorFlow one-hot encoding method here, rather than Scikit-Learn. Scikit-learn would expand these strings to the 0's and 1's as we would typically see for dummy variables.  TensorFlow translates all of the words to index values and replaces each word with that index.

In [10]:
VOCAB_SIZE = 50
encoded_reviews = [one_hot(d, VOCAB_SIZE) for d in reviews]
print(f"Encoded reviews: {encoded_reviews}")

Encoded reviews: [[38, 15, 15], [16, 20], [24, 15], [18, 24], [16, 24], [35], [35, 20], [7], [37, 32], [21, 3, 27, 16]]


The program one-hot encodes these reviews to word indexes; however, their lengths are different.  We pad these reviews to 4 words and truncate any words beyond the fourth word.

In [11]:
MAX_LENGTH = 4

padded_reviews = pad_sequences(encoded_reviews, maxlen=MAX_LENGTH, \
                               padding='post')
print(padded_reviews)

[[38 15 15  0]
 [16 20  0  0]
 [24 15  0  0]
 [18 24  0  0]
 [16 24  0  0]
 [35  0  0  0]
 [35 20  0  0]
 [ 7  0  0  0]
 [37 32  0  0]
 [21  3 27 16]]


Each review is padded by appending zeros at the end, as specified by the padding=post setting.

Next, we create a neural network to learn to classify these reviews. 

In [12]:
model = Sequential()
embedding_layer = Embedding(VOCAB_SIZE, 8, input_length=MAX_LENGTH)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


This network accepts four integer inputs that specify the indexes of a padded movie review.  The first embedding layer converts these four indexes into four vectors of length 8.  These vectors come from the lookup table that contains 50 (VOCAB_SIZE) rows of vectors of length 8. This encoding is evident by the 400 (8 times 50) parameters in the embedding layer. The size of the output from the embedding layer is 32 (4 words expressed as 8-number embedded vectors).  A single output neuron is connected to the embedding layer by 33 weights (32 from the embedding layer and a single bias neuron).  Because this is a single-class classification network, we use the sigmoid activation function and binary_crossentropy.

The program now trains the neural network.  Both the embedding lookup and dense 33 weights are updated to produce a better score.

In [13]:
# fit the model
model.fit(padded_reviews, labels, epochs=100, verbose=0)

We can see the learned embeddings.  Think of each word's vector as a location in 8 dimension space where words associated with positive reviews are close to other words with positive reviews.  Similarly, training places negative reviews close to each other.  In addition to the training setting these embeddings, the 33 weights between the embedding layer and output neuron similarly learn to transform these embeddings into an actual prediction.  You can see these embeddings here.

In [14]:
print(embedding_layer.get_weights()[0].shape)
print(embedding_layer.get_weights())

(50, 8)
[array([[-1.03855960e-01, -5.59105724e-02, -6.04345910e-02,
         1.50698468e-01,  8.98860618e-02,  8.96284208e-02,
         1.65250570e-01,  4.28558849e-02],
       [ 2.30258144e-02,  4.66232412e-02, -3.89982834e-02,
         2.99915113e-02, -2.06652526e-02,  3.44550945e-02,
        -1.44930705e-02, -4.07445915e-02],
       [-4.72828634e-02,  1.85899399e-02,  8.94228369e-03,
         9.16576385e-03, -1.39516592e-03,  3.86941768e-02,
        -2.00331211e-04, -2.19582673e-02],
       [-1.38059288e-01, -6.69398904e-02, -1.37284666e-01,
         8.37527588e-02,  8.94882604e-02,  9.02104154e-02,
         6.08446635e-02,  1.74611673e-01],
       [-3.90188769e-03,  2.26297863e-02, -3.32948118e-02,
        -7.31148571e-03,  3.02187465e-02, -4.14650813e-02,
        -4.23040651e-02,  1.48720481e-02],
       [-2.69240271e-02,  4.56026904e-02, -1.25192292e-02,
        -2.45216247e-02, -4.73748930e-02, -2.03959588e-02,
        -4.59431186e-02,  3.84237655e-02],
       [-1.44931301e-02, 

We can now evaluate this neural network's accuracy, including both the embeddings and the learned dense layer.  

In [15]:
loss, accuracy = model.evaluate(padded_reviews, labels, verbose=0)
print(f'Accuracy: {accuracy}')

Accuracy: 1.0


The accuracy is a perfect 1.0, indicating there is likely overfitting. For a more complex data set, it would be good to use early stopping to not overfit.

In [16]:
print(f'Log-loss: {loss}')

Log-loss: 0.4717821180820465


However, the loss is not perfect, meaning that even though the predicted probabilities indicated a correct prediction in every case, the program did not achieve absolute confidence in each correct answer.  The lack of confidence was likely due to the small amount of noise (previously discussed) in the data set.  Additionally, the fact that some words appeared in both positive and negative reviews contributed to this lack of absolute certainty. 